In [2]:
import numpy as np
import os
import pandas as pd
import keras
import h5py

In [3]:
os.chdir('/content/drive/My Drive/github/NNanobody/')

In [ ]:
new_seeds_all = np.genfromtxt('./data/seeds/filtered_seed.txt', dtype='str')
new_seeds=set([x for x in new_seeds_all if ('X' not in x)])
def check2(x):
    return x in new_seeds
c_seq=new_seeds
check=[]
temp=pd.Series([1]*len(new_seeds))
temp.index=new_seeds
check.append(temp.to_frame('C'))

In [ ]:
check

[                  C
 EYQGRGYPRVLITFDY  1
 DYYRDWPYHRAFDY    1
 EDRRPPLIVFDY      1
 SQYGAYYGFDY       1
 HNPQAQSYLPYRLLDY  1
 ...              ..
 EPGIEDYDMDY       1
 DAFHYQGDRYYFDY    1
 HYQHHLVYVFDY      1
 FEGYLWWGLSQGWFDY  1
 DYWWYSWFDY        1
 
 [6566 rows x 1 columns]]

In [ ]:
for dataset in ['Full Regression', 'Hold out Regression', 'Hold out Top 4%']:
  for model in ['seq_32_32','seq_32x1_16','seq_32x1_16_filt3','seq_32x2_16']:
    for k in [0.005, 0.0005, 0.001, 5e-05, 0.0001]:
      for L in ['10','20', '30', '40']:
        pred_file = f'./generated/{model}/{dataset}_{model}_{k}_{L}.tsv'
        pred_code = f'{dataset}_{model}_{k}_{L}'
        # print(pred_code)
        # smap=pd.read_csv(pred_file,sep='\t',index_col=0,header=None)

        # pred_file = f'sequences/seq_32x2_16/seq_32x2_16_0.005_50.tsv'
        smap=pd.read_csv(pred_file,sep='\t',index_col=0,header=None)
        smap.columns=['new','seed','own_score','sscore']
        smap=smap[smap['seed'].apply(check2)]
        raw_seq=smap['new'].values
        g_seq=np.asarray([x for x in raw_seq if ((x[-2:]=='DY')|(x[-2:]=='DV'))])
        g_seq=np.unique(g_seq)
        temp=pd.Series([1]*len(g_seq))
        temp.index=g_seq
        check.append(temp.to_frame(pred_code))

In [ ]:
combine_check=pd.concat(check,axis=1)
combine_check=combine_check.fillna(0)
count=combine_check[combine_check['C']!=1].sum(axis=1)
final_sorted=count.sort_values(axis=0, ascending=False)
final_sorted

GKRWSRRYVDNRAFDY    218.0
EEQYSAGGNDY         201.0
DNAQVGIFDY          200.0
WYRRNLYSYKPHKFDY    196.0
NPWYWRGFDY          194.0
                    ...  
EHPKHMYHILDY          1.0
DHHEMGRMPMQYDLDY      1.0
EHQQHSQQACREGEDY      1.0
EHQQHSQQMCREGEDY      1.0
WSLDPGTPYIPFDY        1.0
Length: 130405, dtype: float64

In [ ]:
final_sorted

GKRWSRRYVDNRAFDY    218.0
EEQYSAGGNDY         201.0
DNAQVGIFDY          200.0
WYRRNLYSYKPHKFDY    196.0
NPWYWRGFDY          194.0
                    ...  
EHPKHMYHILDY          1.0
DHHEMGRMPMQYDLDY      1.0
EHQQHSQQACREGEDY      1.0
EHQQHSQQMCREGEDY      1.0
WSLDPGTPYIPFDY        1.0
Length: 130405, dtype: float64

In [ ]:
f=open(os.path.join('./generated','data.tsv'),'w')
flabel=open(os.path.join('./generated','data.target'),'w')

pos_seq=list(final_sorted.index)
padlen=20
for i in range(len(pos_seq)):
  justified = pos_seq[i].center(20, 'J')
  f.writelines(str(i)+'\t'+justified+'\n')
  flabel.writelines('1\n')

f.close()
flabel.close()

In [ ]:
data_embed('./generated/data.tsv', './generated/data.target', './generated/data_embedded.h5', 1000000)

data shape:  (130405, 20, 1, 20)


In [ ]:
fi = h5py.File('./generated/data_embedded.h5.batch1', 'r')
test_set = np.asarray(fi['data'])
test_label = np.asarray(fi['label'])

In [ ]:
test_set.shape

(130405, 20, 1, 20)

In [ ]:
def load_all_models():
  model_list = {}
  for dataset in ['Full Regression','Hold out Regression','Hold out Top 4%']:
    for loaded_model in ['seq_32x1_16', 'seq_64x1_16','seq_32x2_16','seq_32_32','seq_32x1_16_filt3','seq_emb_32x1_16']:
      model = keras.models.load_model(f'./weights/regression/{dataset}/{loaded_model}')
      model_list[dataset + ' ' + loaded_model] = model

  return model_list

In [ ]:
models = load_all_models()

In [ ]:
def get_stacked_prediction(model_list):
  output = []
  for idx, model in enumerate(model_list.values()):
    cur_pred = model.predict(test_set).flatten()
    output.append([])
    output[idx].append(cur_pred)
  
  output = np.array(output)
  return output.reshape(output.shape[0], output.shape[2]).T

In [ ]:
output = get_stacked_prediction(models)
stacked_dataset = pd.DataFrame(columns=list(models.keys()), data=output)

In [ ]:
interpreter = keras.models.load_model('./weights/interpreter/interpret_15x1')

In [ ]:
all_predictions = interpreter.predict(stacked_dataset.values)

In [ ]:
all_predictions.shape

(130405, 1)

In [ ]:
stacked_dataset

,Full Regression seq_32x1_16,Full Regression seq_64x1_16,Full Regression seq_32x2_16,Full Regression seq_32_32,Full Regression seq_32x1_16_filt3,Full Regression seq_emb_32x1_16,Hold out Regression seq_32x1_16,Hold out Regression seq_64x1_16,Hold out Regression seq_32x2_16,Hold out Regression seq_32_32,Hold out Regression seq_32x1_16_filt3,Hold out Regression seq_emb_32x1_16,Hold out Top 4% seq_32x1_16,Hold out Top 4% seq_64x1_16,Hold out Top 4% seq_32x2_16,Hold out Top 4% seq_32_32,Hold out Top 4% seq_32x1_16_filt3,Hold out Top 4% seq_emb_32x1_16
0,-0.351805,0.001393,0.168739,-0.361313,-0.349329,-0.592510,-0.372396,-0.499884,-0.940184,-0.646292,-0.863471,-0.891000,-0.124222,-0.028768,0.128610,-0.259990,-0.131335,-0.091968
1,-0.190477,-0.146462,0.563667,-0.368211,-0.224738,-0.539737,-0.854824,-1.031475,-0.931361,-0.468272,-0.867803,-0.863484,-0.274160,0.445194,-0.541297,-0.534864,-0.201252,-0.632249
2,-0.647048,-0.559107,-0.524652,-0.612209,-0.525151,-0.664032,-0.634917,-0.726365,-0.940184,-0.952249,-0.805935,-0.734752,-0.382597,0.072762,-0.412682,-0.688652,-0.682102,-0.379254
3,-0.043234,-0.419705,-0.222908,-0.477776,-0.000199,-0.618681,-0.618138,-0.799116,-0.940184,-0.722401,-0.541764,-0.689156,-0.235912,-0.382883,-0.204379,-0.517829,-0.214029,-0.525811
4,-0.241867,0.059446,-0.232850,-0.488010,0.126378,-0.626434,-0.809190,-0.789852,-0.940184,-0.776602,-0.643998,-0.881152,-0.033553,-0.268791,-0.372883,-0.388978,-0.285045,-0.184782
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1540,0.929195,0.990320,0.795643,0.788479,0.683089,0.970096,0.685583,1.029045,1.155982,0.833124,0.825595,0.556875,0.337187,0.823547,0.860151,0.033830,0.524408,0.243493
1541,0.507704,0.556147,0.204120,0.223951,0.242911,0.049155,0.320482,0.565478,0.003297,0.454521,0.370866,0.176795,-0.110205,0.142390,0.228253,-0.315346,-0.170931,0.017997
1542,0.784571,0.883432,0.583509,0.117606,0.456884,0.194327,0.430117,0.754105,0.329682,0.476595,0.294965,0.141726,0.042211,0.834979,0.599045,-0.482193,0.302406,-0.146600
1543,0.043391,-0.166475,0.270870,-0.583827,-0.720096,-0.362062,0.376316,0.330412,-0.940184,-0.835206,-0.898120,-0.529728,-0.612337,-0.270302,-0.228977,-0.723806,-0.730213,-0.634873


In [ ]:
# all_score=pd.read_csv(os.path.join('./sequences','scores.csv'),index_col=0)
# all_score=all_score.loc[final_sorted.index]
# reg_score=all_score[all_score.columns[0:-5]]
# new_gifford=final_sorted.to_frame('vote')
# new_gifford['cnn_score']=reg_score.mean(axis=1)

new_gifford=final_sorted.to_frame('vote')

new_gifford['scores'] = all_predictions
new_gifford['cnn_ci']=(stacked_dataset.std(axis=1)).values*1.96*2/np.sqrt(len(stacked_dataset.columns))
new_gifford['5%_Gauss_lb']=new_gifford['scores']-(new_gifford['cnn_ci'])/2
# new_gifford['R3/R1J_score']=gifford2['R3/R1J_score'].loc[new_gifford.index]
# new_gifford['R3/R1_score']=gifford2['R3/R1_score'].loc[new_gifford.index]
new_gifford['Ens_Grad2']=True
new_gifford['Ens_Grad2'][new_gifford['5%_Gauss_lb']<0]=False
new_gifford['Ens_Grad2'][new_gifford['vote']<10]=False
top=new_gifford[new_gifford['Ens_Grad2']>0].index
# set1=set(gifford2[gifford2['Seed_new']==1].index)
set3=set(top)
set3

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  from ipykernel import kernelapp as app
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  app.launch_new_instance()


{'GKRWSRRYGDMRDFDY',
 'GKRWAARYGDRRAFDY',
 'QYDARHDYHEGMDY',
 'HKPQAGFYLPYRILDY',
 'HKPQAISYLTYRLLDY',
 'DHGPHRDPYWPWIFDY',
 'DLFRYYYYFNPLDY',
 'DHGTQRDPYWPWKFDY',
 'HKPQAGSYLTYRIMDY',
 'VRGGHDFEARVHDY',
 'GHYYDIMVFPWDTFDY',
 'HKPQSWSYLPYRILDY',
 'HKPQARSYLPYIILDY',
 'WTLGPGTPYIPFDY',
 'DIKTESYPSYHYYFDY',
 'HKPQAMSYVPYRLLDY',
 'LHYYDIGVFPWDTFDY',
 'HKPHAKSYLPYQILDY',
 'QKPQAQSYLPYRILDY',
 'VRGGCEFEKRVHDY',
 'DICWDSYPSYHYYFDY',
 'GKHWSRRYMDRRAFDY',
 'DLFLYSYFMWPLDY',
 'PSSTPFNHWAYWGFDY',
 'GKRMARRYGDRRAFDY',
 'DHGPHRDPYWPWMFDY',
 'WRLDPGTPYIPFDY',
 'FPGLYHAMKFWPYFDY',
 'MSNGHEFEKRVHDY',
 'PTQIWDYHCHWYDFDY',
 'GKRWSRQSGDRRAFDY',
 'HKPQAKSYLPNRLLDY',
 'SWQSEGSYWEIKGFDY',
 'DCGFHDQDQSHYMDY',
 'GKRKSRRYGDRRAVDY',
 'WRLGPGTPYIPFDY',
 'IKPQATSYLPYRLLDY',
 'EPWYWRGFDY',
 'GPWYWRDFDY',
 'GKRGRRRYGDRRAFDY',
 'DLAYWYVRGDGLGFDY',
 'GKRWSRMSGDRRAFDY',
 'PSSTPFWHWMYWGFDY',
 'GKRWSRRYIYRRAFDY',
 'PQQWSGYPRQKYSFDY',
 'HMPQAKSYLPYRLLDY',
 'GKRWPRRYNDRRAFDY',
 'EPWYYRGLDY',
 'GAWDPETPYIPFDY',
 'HKPQAMSY

In [ ]:
new_gifford.to_csv('ranked_sequences.csv')

In [4]:
new_gifford = pd.read_csv('./generated/ranked_sequences.csv')

In [10]:
top=new_gifford[new_gifford['Ens_Grad2']>0]
top.to_csv('top_sequences.csv')